In [252]:
molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1)
ent_type = torch.tensor(np.load(f'{data_path}/ent_type_onehot.npy'))

nodes, _, _, edge_mask = k_hop_subgraph(0, 1, G_tg.edge_index)
double_mask = molecule_mask * edge_mask
mask_idx = torch.where(double_mask)[0]
edge_subgraph = G_tg.edge_subgraph(mask_idx)
subgraph = edge_subgraph.subgraph(nodes)
masked_node_ids = edge_subgraph.edge_index[0] # (num_masked_nodes,)
motif_labels = motifs[masked_node_ids] # (num_masked_nodes, motif_len)
node_labels = ent_type[masked_node_ids] # (num_masked_nodes, num_ent_type)
rel_labels = subgraph.relation

subgraph, masked_node_ids.shape, motif_labels.shape, node_labels.shape, rel_labels.shape

(Data(edge_index=[2, 1951], relation=[1951], num_nodes=385),
 torch.Size([1951]),
 torch.Size([1951, 85]),
 torch.Size([1951, 16]),
 torch.Size([1951]))

In [251]:
rel_labels

tensor([ 0,  0,  0,  ..., 31, 31, 31])

In [238]:
print(G_tg.edge_index[:, mask_idx][0])

tensor([    0,     0,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,    29,    29,    29,    29,   661,   661,   661,   661,   661,
          661,   661,   661,   661,   661,  2277,  2343,  2343,  2343, 18686,
        18686, 18686, 18686, 18686, 18686, 18686, 28000, 28000, 28000, 28749,
        29943, 29943, 29943, 29943, 29943, 29943, 33784, 33784, 33784, 33784,
        33784, 39941, 39941, 39941, 39941, 39941, 39941, 42393, 42393, 42393,
        42393, 42393, 42393, 42393, 42393, 42393, 42624, 42624, 42624, 42624,
        42624, 42624, 43274, 43462, 48532, 48532, 48532, 48532, 48532, 48532,
        58132, 58132, 58132, 58132, 58132, 58132, 58132, 60341, 65188])


In [239]:
subgraph.edge_index

tensor([[ 0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,
          5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  8,  9,  9,  9,  9,
          9,  9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12,
         12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 15, 16, 16, 16, 16, 16, 16,
         17, 17, 17, 17, 17, 17, 17, 18, 19],
        [ 1,  7,  0,  2,  4,  5,  3, 18,  8,  6, 14, 11, 16,  7, 15, 13,  9, 10,
         17, 19, 12,  1,  2,  5,  3,  1,  2, 16, 11, 12, 10,  6, 13,  9, 17,  1,
          1,  2,  7,  1,  3, 13,  9, 17, 12, 11,  0,  1,  5,  1,  1,  3,  6, 13,
         17, 12,  1,  3, 11, 16, 12,  1,  3,  6, 10, 16, 12,  1,  3,  6,  9, 10,
         11, 16, 13, 17,  1,  3,  6,  9, 12, 17,  1,  1,  1,  3, 10, 11, 12, 17,
          1,  3,  6,  9, 12, 13, 16,  1,  1]])

In [107]:
s1.number_of_nodes(), s1.number_of_edges()

(3433, 3432)

In [94]:
for c_id in tqdm(center_molecule_ids):
    subgraph = s1 = get_subgraph(G, c_id)
    

16979

In [1]:
from pretrain import *

# your credentials

params = {
    "lr": 1e-4,
    "hidden_dim": 200,
    "epochs": 100,
    "lambda": "08_15_10"
}
logger = get_logger(lr=params['lr'], hidden_dim=params['hidden_dim'], epochs=params['epochs'], lambda_=params['lambda'])


# Data path
data_path = '../data_process/pretrain_data'
print('Getting everything prepared...')
ent_type, motifs, G_tg, center_molecule_ids, molecule_mask = get_everything(data_path)

# Load KGE embeddings
# return:
# entity_embedding: (num_ent, emb_dim)
# relation_embedding: (num_rel, emb_dim)
emb_path = '/data/pj20/molkg_kge/transe'
print('Loading KGE embeddings...')
entity_embedding, relation_embedding = load_kge_embeddings(emb_path)

# Initialize model
print('Initializing model...')
model = KGNN(
    node_emb=entity_embedding,
    rel_emb=relation_embedding,
    num_nodes=ent_type.shape[0],
    num_rels=39,
    embedding_dim=512,
    hidden_dim=200,
    num_motifs=motifs.shape[1],
    lambda_edge=0.8,
    lambda_motif=1.5,
    lambda_mol_class=1
)

# Train
device = torch.device('cuda:1')
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 100

/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Getting everything prepared...
Loading entity type labels...
Loading center molecule motifs...
Loading entire knowledge graph...


/home/pj20/miniconda3/envs/kgc/lib/python3.8/site-packages/torch_geometric/utils/convert.py:250: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  data[key] = torch.tensor(value)
/home/pj20/gode/k_level_pretrain/pretrain.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  molecule_mask = torch.tensor(ent_type[:,0][G_tg.edge_index[0]] == 1) # (num_edges,)


Loading molecule mask...
Loading KGE embeddings...
Loading KGE embeddings...
Initializing model...


In [3]:
# Get dataloader
train_loader, val_loader = get_dataloader(G_tg, center_molecule_ids, molecule_mask, motifs, ent_type, batch_size=4)
train_loop(model, train_loader, val_loader, optimizer, device, epochs)

loss: 0:   0%|          | 0/7605 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [4]:
for batch in train_loader:
    print(batch)
    break

DataBatch(edge_index=[2, 122], relation=[122], num_nodes=22, masked_node_ids=[122], rel_ids=[122], center_molecule_id=[1], motif_labels=[122, 85], node_labels=[122, 16], batch=[22], ptr=[2])


In [4]:
torch.zeros((1, 39)).shape

torch.Size([1, 39])

In [6]:
torch.zeros(39).reshape(1, -1).shape

torch.Size([1, 39])

In [13]:
torch.where(torch.tensor([0, 1, 4, 5]) == 4)[0][0].int()

tensor(2, dtype=torch.int32)